In [1]:
api_key='yourkey'
secret_key='yourkey'

In [2]:
# Import libraries
import pandas as pd
import json
import requests
import math

from alpaca.data import CryptoHistoricalDataClient, StockHistoricalDataClient
from alpaca.data.requests import CryptoBarsRequest
from alpaca.data.timeframe import TimeFrame
from datetime import datetime
from alpaca.data.requests import StockLatestQuoteRequest
from alpaca.data.historical import CryptoHistoricalDataClient
from alpaca.data.requests import CryptoLatestQuoteRequest
import pandas as pd
import numpy as np

In [3]:
cryptos=('BTC','ETH','XRP','ADA','DOGE','SOL','DOT','SHIB','DASH','LTC','AVAX','TRX','UNI','ATOM','LINK','XMR','BCH','XLM','NEAR','ALGO','VET','HBAR','FTM','AAVE','MANA','EOS','EGLD','XTZ','CHZ','GRT','FXS','ADA','NEO')
investment=100
cryptos1=('BTC','ETH','AAVE', 'ALGO', 'BAT', 'BCH', 'BTC', 'DAI', 'GRT', 'LINK', 'LTC', 'MATIC', 'MKR', 'NEAR', 'TRX', 'UNI', 'SOL')
string='/USD'
cryptos_dig=[x + string for x in cryptos1]

In [4]:
def best(crypto,investment):
    def price(crypto):
        crypto=crypto+"USDT"
        key = "https://api.binance.com/api/v3/ticker/price?symbol="+ crypto
        data = requests.get(key)
        data = data.json()
        price=data['price']
        return (price)
    p=float(price(crypto))
    step_sizes_list = pd.read_csv(r'C:\Users\pablosilva\Documents\Archivos Temporales\Step Size Binance.csv')
    step_sizes_list.head(2)
    for i,row in step_sizes_list.iterrows():
        if row['Trading Pair']==crypto+"USDT":
            step=row['Updated Step Size']
        elif row['Trading Pair']==crypto+"BUSD":
            step=row['Updated Step Size']
        elif row['Trading Pair']==crypto+"USDC":
            step=row['Updated Step Size']
        elif row['Trading Pair']==crypto+"USD":
            step=row['Updated Step Size']
        else:
            pass
    def round_decimals_down(number:float):
        decimals=int(len(str(step))-2)
        """
        Returns a value rounded down to a specific number of decimal places.
        """
        if not isinstance(decimals, int):
            raise TypeError("decimal places must be an integer")
        elif decimals < 0:
            raise ValueError("decimal places has to be 0 or more")
        elif decimals == 0:
            return math.floor(number)

        factor = 10 ** decimals
        return math.floor(number * factor) / factor
    buy_qty=investment/p
    fee=buy_qty*0.001
    cost=buy_qty*p
    position=buy_qty-fee
    qty_sell=round_decimals_down(position)
    fee_usd=fee*p
    reminder_val=(buy_qty-qty_sell)*p
    increment=(reminder_val+(fee_usd))/cost
    return crypto, increment,p,position

In [5]:
increments=[]
for i in cryptos:
    increments.append(best(i,investment))
    df=pd.DataFrame(increments)
    df.columns=['Crypto','BE_INC','Price','Position']
    pd.set_option('display.float_format', lambda x: '%0.4f' % x)
    df.sort_values(by=['BE_INC'],inplace=True)

In [6]:
def daily(symbol):
    client = CryptoHistoricalDataClient()

    request_params = CryptoBarsRequest(
                                symbol_or_symbols=symbol,
                                timeframe=TimeFrame.Day,
                                start="2022-01-01 05:00:00",
                                end="2023-01-30 05:00:00"
                         )

    bars = client.get_crypto_bars(request_params)
    df1=bars.df
    df1['variation']=0.0
    df1['weight']=0.0
    for i,row in df1.iterrows():
        df1.loc[i, "variation"]= row["high"]-row["low"]
    for i,row in df1.iterrows():
        df1.loc[i, "weight"]= row["variation"]/((row["high"]+row["low"])/2)
    mean_vol=df1['trade_count'].mean()
    mean=df1['weight'].mean()
    return symbol,mean,mean_vol

In [11]:
daily_r=[]
for i in cryptos_dig:
    daily_r.append(daily(i))
    df2=pd.DataFrame(daily_r)
    df2.columns=['Pair','D_Volatility','Trade_Count']
    pd.set_option('display.float_format', lambda x: '%.4f' % x)
    df3=df2.sort_values(by='D_Volatility', ascending=False)

In [8]:
df.head(5)

,Crypto,BE_INC,Price,Position
7,SHIB,0.0020,0.0000,8575107.2961
11,TRX,0.0020,0.0630,1585.2110
20,VET,0.0020,0.0235,4251.0638
4,DOGE,0.0020,0.0885,1128.4310
3,ADA,0.0020,0.3794,263.3105


In [12]:
df3.head(5)

,Pair,D_Volatility,Trade_Count
16,SOL/USD,0.0945,13538.0759
8,GRT/USD,0.0935,1613.8278
2,AAVE/USD,0.0891,1456.3519
13,NEAR/USD,0.0878,718.8838
15,UNI/USD,0.0849,2059.8152
